# NNMD of ammonia using the pretrained models

In this notebook, we perform a NN-based MD NVE simulation using ASE and the [SchNet NN potential](https://github.com/learningmatter-mit/NeuralForceField). We will be using the third generation of ammonia models, as shown in [our paper](https://arxiv.org/abs/2101.11588).

The utilities at `nff` will be used to perform the MD simulation. `nglview` will be used to visualize the generated trajectories.

In [1]:
import os
import numpy as np
from ase import Atoms, units
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from nff.md.nve import Dynamics
from nff.md.nvt import NoseHoover
from nff.data import Dataset
import nff.utils.constants as const

import nglview as nv
from ase.io import Trajectory, read

## Loading the models and parameters

The dataset contains a PyTorch dataset with all the DFT data used to train the third generation of NN potentials. Here, we will use the pre-trained ensemble. For learning how to train the models using the SchNet architecture and the current dataset, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [2]:
dset = Dataset.from_file('../data/ammonia.pth.tar')

`DEVICE` sets the GPU used for evaluating the model. If you want to evaluate on a CPU, set `DEVICE = 'cpu'`. The models are stored at `/models/ammonia`, where `/` is the root folder of this repository.

In [43]:
DEVICE = 1

def get_ensemble_calc(device=DEVICE):
    PATH = '../models/ammonia'
    models = []
    for model_name in sorted(os.listdir(PATH)):
        m = NeuralFF.from_file(os.path.join(PATH, model_name), device=device).model
        models.append(m)

    return EnsembleNFF(models, device=device)

ensemble = get_ensemble_calc(device=DEVICE)

Here, we set the parameters for the MD simulation. For learning how to use these parameters within our code, check the tutorials at the original [NFF repo](https://github.com/learningmatter-mit/NeuralForceField).

In [44]:
def get_md_params(traj_filename, temperature=1000):
    return {
        'T_init': temperature,
        'time_step': 1.0,
        'thermostat': VelocityVerlet,  
        'thermostat_params': {'timestep': 1.0 * units.fs},
        'steps': 5 * 2000,
        'save_frequency': 40,
        'nbr_list_update_freq': 10,
        'thermo_filename': 'thermo.log',
        'traj_filename': traj_filename,
        'skip': 0
    }

Finally, we use the lowest energy conformation within the existing dataset (i.e. the ground state of ammonia) as a starting configuration for the MD simulation. `AtomsBatch` is a [wrapper within our NFF repo](https://github.com/learningmatter-mit/NeuralForceField/blob/master/nff/io/ase.py) and can be used to interface an ASE atom with NFF.

In [49]:
CUTOFF = 5.0

def get_md_atoms(dset=dset, cutoff=CUTOFF, device=DEVICE):
    props = dset[np.argmin(dset.props['energy'])]

    atoms = AtomsBatch(
        positions=props['nxyz'][:, 1:],
        numbers=props['nxyz'][:, 0],
        cutoff=cutoff,
        props={'energy': 0, 'energy_grad': []},
        calculator=ensemble,
        device=device,
    )
    _ = atoms.update_nbr_list()
    
    return atoms

## Performing the MD simulation

Now, we perform the MD simulation using the parameters shown before.

In [53]:
TEMPERATURE = 1500

name = f'NVE_{TEMPERATURE}.traj'

atoms = get_md_atoms()
md_params = get_md_params(name, temperature=TEMPERATURE)
dyn = Dynamics(atoms, md_params)
traj = dyn.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.5052      -0.0017       0.5069   980.4

0.0400           0.5213       0.3041       0.2171   420.0

0.0800           0.5326       0.4968       0.0358    69.2

0.1200           0.5131       0.1409       0.3722   719.9

0.1600           0.5181       0.1654       0.3527   682.2

0.2000           0.5246       0.2395       0.2850   551.3

0.2400           0.5098       0.1369       0.3729   721.3

0.2800           0.5346       0.4025       0.1322   255.7

0.3200           0.5055       0.1574       0.3481   673.2

0.3600           0.5373       0.4413       0.0960   185.6

0.4000           0.5067       0.2343       0.2725   527.0

0.4400           0.5261       0.3942       0.1319   255.1

0.4800           0.5195       0.3362       0.1833   354.6

0.5200           0.5080       0.2404       0.2676   517.6

0.5600           0.5333       0.4018       0.1316   254.5

0.6000           0.5059       0.1630       0.3429   663.1


6.2400           0.5159       0.1390       0.3769   729.0

6.2800           0.5241       0.2333       0.2907   562.3

6.3200           0.5125       0.1080       0.4045   782.4

6.3600           0.5305       0.3099       0.2205   426.6

6.4000           0.5115       0.0776       0.4339   839.2

6.4400           0.5312       0.2839       0.2473   478.3

6.4800           0.5100       0.0578       0.4522   874.6

6.5200           0.5316       0.2851       0.2465   476.8

6.5600           0.5103       0.0920       0.4184   809.1

6.6000           0.5289       0.2579       0.2710   524.2

6.6400           0.5099       0.1389       0.3710   717.6

6.6800           0.5210       0.2001       0.3209   620.7

6.7200           0.5166       0.1843       0.3323   642.7

6.7600           0.5105       0.0901       0.4205   813.2

6.8000           0.5303       0.2575       0.2728   527.5

6.8400           0.5077       0.0534       0.4544   878.8

6.8800           0.5357       0.3348       0.2010   388.

## Visualizing the trajectory

Finally, we can visualize the trajectory on the Jupyter server using `nglview`. The filename was selected prior to the MD simulation.

In [54]:
filetraj = Trajectory(name)
view = nv.show_asetraj(filetraj)
view

NGLWidget(max_frame=250)